**By using the output of this notebook, you are accepting the [competition rules](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/rules).**


## References

- Monochrome fix and scaling: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
- Resizing and saving image: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image

In [1]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import cv2

In [2]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [3]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [4]:
def enh_alphaTMean(im,alpha,n=5):
    img = np.zeros(im.shape,dtype=np.int16)
    
    v = (n-1)/2
    
    # Calculate the trim coefficient
    b = int((n*n)*(alpha))
    
    # Process the image
    for i in range(0,im.shape[0]):
        for j in range(0,im.shape[1]):
            # Extract the window area
            block = im[int(max(i-v,0)):int(min(i+v+1,im.shape[0])), int(max(j-v,0)):int(min(j+v+1,im.shape[1]))]

            # Reshape the neighborhood into a vector by flattening the 2D block
            wB = block.flatten()
            
            # Sort the vector into ascending order
            wB = np.sort(wB)
            len = wB.size
            
            # Trim b elements from each end of the vector
            if (b != 0):
                nwB = wB[b:len-b]
    
            # Calculate the mean of the trimmed vector
            tMean = nwB.mean()

            # Assign the values
            if (tMean > 0):
                img[i][j] = int(tMean)
    return img

def alpha_filter(image,alpha):
    alpha = float(alpha)
    result_img = enh_alphaTMean(image,alpha,5)
    return result_img

In [5]:
def preprocess(img):    
    img = np.array(img)
    
    
    img = cv2.equalizeHist(img) #Equalize histogram
    
    img = cv2.GaussianBlur(img, (3,3), 10) #Gaussian Filter
    
#     img = alpha_filter(img, 6.0) #Alpha-Trimmed Mean Filter
    
    kernel = cv2.getGaborKernel((9, 9), 1.0, np.pi/4, 1.0, 0.5, 0, ktype=cv2.CV_32F)
    img = cv2.filter2D(img, cv2.CV_8UC3, kernel) # Gabor Filte
    
    
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    img = cv2.normalize(img,  norm_img, 0, 255, cv2.NORM_MINMAX)
    
    
    # Adaptive Median Filter, Alpha-Trimmed Mean Filter, Gaussian Filter,
    # Gabor Filter, High Pass Filter, Laplacian Filter, and Bilateral Filter. H.
    # Kim et al. [20] used the Median Filter (the kernel size is 3 × 3) and
    # Top-hat filter to reduce image noise and to smooth it
    return Image.fromarray(img)

In [6]:
image_id = []
dim0 = []
dim1 = []

for split in ['train', 'test']:
    load_dir = f'../{split}/'
    save_dir = f'./kaggle/tmp/{split}/'

    os.makedirs(save_dir, exist_ok=True)

    for file in tqdm(os.listdir(load_dir)):
        # set keep_ratio=True to have original aspect ratio
        xray = read_xray(load_dir + file)
        im = resize(xray, size=256)
        im = preprocess(im)
        im.save(save_dir + file.replace('dicom', 'png'))
        
        if split == 'train':
            image_id.append(file.replace('.dicom', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])

  0%|          | 0/15000 [00:00<?, ?it/s]

/home/weronika/.local/lib/python3.8/site-packages/pydicom/pixel_data_handlers/pillow_handler.py:176: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(
/home/weronika/.local/lib/python3.8/site-packages/pydicom/pixel_data_handlers/pillow_handler.py:176: UserWarning: The (0028,0101) 'Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


  0%|          | 0/3000 [00:00<?, ?it/s]

In [7]:
%%time
!tar -zcf train.tar.gz -C "./kaggle/tmp/train/" .
!tar -zcf test.tar.gz -C "./kaggle/tmp/test/" .

CPU times: user 252 ms, sys: 56.1 ms, total: 308 ms
Wall time: 22.9 s


In [8]:
df = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1})
df.to_csv('train_meta.csv', index=False)